# MNIST GAN

## Obtencion del dataset

In [1]:
#!rm -rf data
#!git clone https://github.com/juanma1982/GANNets.git data

In [2]:
!ls -l data/mnist_png/training/0 | head

total 23692
-rw-r----- 1 gaston gaston 242 dic 10  2015 10005.png
-rw-r----- 1 gaston gaston 280 dic 10  2015 1000.png
-rw-r----- 1 gaston gaston 174 dic 10  2015 10010.png
-rw-r----- 1 gaston gaston 287 dic 10  2015 10022.png
-rw-r----- 1 gaston gaston 286 dic 10  2015 10025.png
-rw-r----- 1 gaston gaston 313 dic 10  2015 10026.png
-rw-r----- 1 gaston gaston 281 dic 10  2015 10045.png
-rw-r----- 1 gaston gaston 280 dic 10  2015 10069.png
-rw-r----- 1 gaston gaston 309 dic 10  2015 10071.png
ls: write error: Broken pipe


## Imports & constants

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import numpy as np
import tensorflow as tf

In [4]:
TRAINING_DATA_DIR = os.path.join("data", "mnist_png", "training")
CHECKPOINT_DIR = "checkpoints"

# Number of inputs counting both mnist data and generated data for the discriminator, and number of random inputs for
# the generator
BATCH_SIZE = 60

GEN_LEARNING_RATE = 1e-4
DISC_LEARNING_RATE = 1e-4

LATENT_SPACE_SHAPE = 100

GEN_VARIABLE_SCOPE = "generator"
DISC_VARIABLE_SCOPE = "discriminator"

MAX_STEPS = 1000000

## Funciones del modelo y entrenamiento

In [5]:
def generator(latent_space, label, training=True):
    """
    Defines the generator network using the latent_space as input.
    Args:
        latent_space: input for the generator network
        label: 10 dimensioanl one-hot tensor
    Returns:
        Generated images
    """
    with tf.variable_scope(GEN_VARIABLE_SCOPE):
        net = tf.concat([latent_space, label], axis=1)
        net = tf.layers.dense(net, 7 * 7 * 64, activation=None, use_bias=False)
        net = tf.layers.batch_normalization(net, training=training)

        # 7 x 7
        net = tf.reshape(net, [-1, 7, 7, 64])

        # 7 x 7
        net = tf.layers.conv2d_transpose(net, 64, kernel_size=(5, 5),
                                         strides=(1, 1),
                                         activation=None,
                                         padding='same',
                                         use_bias=False)
        net = tf.layers.batch_normalization(net, training=training)
        net = tf.nn.leaky_relu(net)

        # 14 x 14
        net = tf.layers.conv2d_transpose(net, 32, kernel_size=(5, 5),
                                         strides=(2, 2),
                                         activation=None,
                                         padding='same',
                                         use_bias=False)
        net = tf.layers.batch_normalization(net, training=training)
        net = tf.nn.leaky_relu(net)

        # 28 x 28
        images = tf.layers.conv2d_transpose(net, 1, kernel_size=(5, 5),
                                            strides=(2, 2),
                                            activation=tf.nn.sigmoid,
                                            padding='same',
                                            use_bias=False)
        return images


def discriminator(images, label, training=True):
    """Defines the discriminator network
    Args:
        images: input images as 28x28 tensors
        label: 10 dimensioanl one-hot tensor
    Returns:
        Logits and prediction for each image
    """
    with tf.variable_scope(DISC_VARIABLE_SCOPE, reuse=tf.AUTO_REUSE):
        net = images
        net = tf.layers.conv2d(net, 64, kernel_size=(5, 5), strides=(2, 2),
                               activation=tf.nn.leaky_relu, padding='same')
        net = tf.layers.dropout(net, training=training)

        net = tf.layers.conv2d(net, 128, kernel_size=(5, 5), strides=(2, 2),
                               activation=tf.nn.leaky_relu, padding='same')
        net = tf.layers.dropout(net, training=training)

        net_shape = net.shape
        net_reshaped = tf.reshape(net, [-1,
                                        net_shape[1] * net_shape[2] * net_shape[
                                            3]])
        net_with_label = tf.concat([net_reshaped, label], axis=1)
        logits = tf.layers.dense(net_with_label, 1,
                             activation=None)

        return logits


def _parse_function(filename, label):
    """
    Reads an image from a file, decodes it into a dense tensor, and resizes it to a fixed shape.
    """
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_png(image_string)
    image_resized = tf.reshape(image_decoded, [28, 28, 1])
    return tf.cast(image_resized, tf.float32) / 255, label


def _mnist_filenames_and_labels():
    """
    Returns:
        A tuple of lists, where the first list contains the mnist png file paths, and the second list contains the
        label for each image.
    """
    images_paths = []
    images_labels = []
    for label in range(10):
        images_dir = os.path.join(TRAINING_DATA_DIR, str(label))
        current_images_paths = os.listdir(images_dir)
        images_paths += list(
            map(lambda image_path: os.path.join(images_dir, image_path),
                current_images_paths))
        images_labels += [label] * len(current_images_paths)
    return images_paths, images_labels


def shuffle(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
  
def _generator_step(sess):
    latent_space_np = np.random.randn(BATCH_SIZE, LATENT_SPACE_SHAPE)
    label = np.random.randint(10, size=BATCH_SIZE)
    _, G_loss_np, step_value = sess.run([G_optimizer, G_loss, step],
                            feed_dict={latent_space: latent_space_np,
                                       G_label: label})
    if step_value % 97 == 0:
        print()
        print("Step: ", sess.run(step))
        print("G_loss: ", G_loss_np)


def _discriminator_step(sess):
    latent_space_np = np.random.randn(BATCH_SIZE // 2, LATENT_SPACE_SHAPE)
    label = np.random.randint(10, size=BATCH_SIZE // 2)
    _, D_loss_np, step_value = sess.run([D_optimizer, D_loss, step],
                            feed_dict={latent_space: latent_space_np,
                                       G_label: label})
    if step_value % 97 == 0:
        print()
        print("Step: ", sess.run(step))
        print("D_loss: ", D_loss_np)

## Definición del grafo de TF y ejecución de la Session

In [6]:
filenames, labels = _mnist_filenames_and_labels()
filenames, labels = shuffle(np.array(filenames), np.array(labels))

step = tf.train.get_or_create_global_step()

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(_parse_function)
dataset = dataset.shuffle(buffer_size=1000)

# The other half of the batch will come from the generator.
dataset = dataset.batch(batch_size=BATCH_SIZE // 2)

dataset = dataset.repeat()

iterator = dataset.make_one_shot_iterator()

# Iterator for tuples of images and labels
next = iterator.get_next()

latent_space = tf.placeholder(tf.float32, shape=[None, LATENT_SPACE_SHAPE])
G_label = tf.placeholder(tf.int32, shape=[None])
G_label_one_hot = tf.one_hot(G_label, 10)
G_images = generator(latent_space, G_label_one_hot)

D_fake_logits = discriminator(G_images, G_label_one_hot)

real_image = next[0]
real_label = tf.one_hot(next[1], 10)
D_real_logits = discriminator(real_image, real_label)

G_expected = tf.ones_like(D_fake_logits)
G_loss = tf.losses.sigmoid_cross_entropy(G_expected, D_fake_logits)

D_real_expected = tf.ones_like(D_real_logits)
D_fake_expected = tf.zeros_like(D_fake_logits)

D_real_loss = tf.losses.sigmoid_cross_entropy(D_real_expected, D_real_logits)
D_fake_loss = tf.losses.sigmoid_cross_entropy(D_fake_expected, D_fake_logits)
D_loss = D_real_loss + D_fake_loss

G_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                GEN_VARIABLE_SCOPE)

G_optimizer = tf.train.AdamOptimizer(
    learning_rate=GEN_LEARNING_RATE).minimize(G_loss, var_list=G_variables,
                                              global_step=tf.train.get_global_step())

D_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                DISC_VARIABLE_SCOPE)

D_optimizer = tf.train.AdamOptimizer(
    learning_rate=DISC_LEARNING_RATE).minimize(D_loss, var_list=D_variables,
                                               global_step=tf.train.get_global_step())

tf.summary.scalar("Gen loss", G_loss, family="Generator")
tf.summary.scalar("Disc loss", D_loss, family="Discriminator")
tf.summary.image("Gen images", G_images, max_outputs=8)


hooks = [tf.train.StopAtStepHook(num_steps=MAX_STEPS)]

with tf.train.MonitoredTrainingSession(checkpoint_dir=CHECKPOINT_DIR,
                                       hooks=hooks) as sess:
    while not sess.should_stop():
        _generator_step(sess)
        _discriminator_step(sess)

INFO:tensorflow:Summary name Gen loss is illegal; using Gen_loss instead.
INFO:tensorflow:Summary name Disc loss is illegal; using Disc_loss instead.
INFO:tensorflow:Summary name Gen images is illegal; using Gen_images instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into checkpoints/model.ckpt.

Step:  1
G_loss:  0.69694316

Step:  98
D_loss:  0.50812197
INFO:tensorflow:global_step/sec: 32.9906

Step:  195
G_loss:  3.118739
INFO:tensorflow:global_step/sec: 52.2267

Step:  292
D_loss:  0.103111
INFO:tensorflow:global_step/sec: 51.8446

Step:  389
G_loss:  3.6806061
INFO:tensorflow:global_step/sec: 49.7439

Step:  486
D_loss:  0.08275227
INFO:tensorflow:global_step/sec: 53.6568

Step:  583
G_loss:  2.6488984
INFO:tensorflow:global_step/sec: 54.1436

Step:  680
D_loss:  0.22402844
INFO:tensorflow:global_step/sec: 51.1632

Ste

INFO:tensorflow:global_step/sec: 50.0041

Step:  10574
D_loss:  1.3345567
INFO:tensorflow:global_step/sec: 49.9175

Step:  10671
G_loss:  1.0231185
INFO:tensorflow:global_step/sec: 49.9645

Step:  10768
D_loss:  1.4364579
INFO:tensorflow:global_step/sec: 49.8897

Step:  10865
G_loss:  1.2854505
INFO:tensorflow:global_step/sec: 49.8486

Step:  10962
D_loss:  1.3362939
INFO:tensorflow:global_step/sec: 49.7786

Step:  11059
G_loss:  1.0417672
INFO:tensorflow:global_step/sec: 50.9114

Step:  11156
D_loss:  1.0266836
INFO:tensorflow:global_step/sec: 53.9082

Step:  11253
G_loss:  0.94744074
INFO:tensorflow:global_step/sec: 49.3114

Step:  11350
D_loss:  1.1229856
INFO:tensorflow:global_step/sec: 52.4456

Step:  11447
G_loss:  1.4769137
INFO:tensorflow:global_step/sec: 55.6747

Step:  11544
D_loss:  1.2661712
INFO:tensorflow:global_step/sec: 54.6089

Step:  11641
G_loss:  1.018965
INFO:tensorflow:global_step/sec: 55.5747

Step:  11738
D_loss:  1.1473033
INFO:tensorflow:global_step/sec: 55.51


Step:  21438
D_loss:  1.4020047
INFO:tensorflow:global_step/sec: 55.8893

Step:  21535
G_loss:  0.8592949
INFO:tensorflow:global_step/sec: 56.1865

Step:  21632
D_loss:  1.2088805
INFO:tensorflow:global_step/sec: 56.085

Step:  21729
G_loss:  0.969383
INFO:tensorflow:global_step/sec: 56.1208

Step:  21826
D_loss:  1.2374046
INFO:tensorflow:global_step/sec: 55.9125

Step:  21923
G_loss:  0.7191667
INFO:tensorflow:global_step/sec: 55.0964

Step:  22020
D_loss:  1.4761686
INFO:tensorflow:global_step/sec: 55.9066

Step:  22117
G_loss:  0.74423844
INFO:tensorflow:global_step/sec: 56.1432

Step:  22214
D_loss:  1.1705437
INFO:tensorflow:global_step/sec: 55.9875

Step:  22311
G_loss:  0.8086605
INFO:tensorflow:global_step/sec: 56.0045

Step:  22408
D_loss:  1.3728386
INFO:tensorflow:global_step/sec: 55.9719

Step:  22505
G_loss:  0.9467593
INFO:tensorflow:global_step/sec: 52.8765

Step:  22602
D_loss:  1.2198708

Step:  22699
G_loss:  0.90719974
INFO:tensorflow:global_step/sec: 53.7813

Step

INFO:tensorflow:global_step/sec: 56.3195

Step:  32302
D_loss:  1.2123518

Step:  32399
G_loss:  0.83456725
INFO:tensorflow:global_step/sec: 56.1873

Step:  32496
D_loss:  1.2782269
INFO:tensorflow:global_step/sec: 56.0815

Step:  32593
G_loss:  1.0222814
INFO:tensorflow:global_step/sec: 56.1009

Step:  32690
D_loss:  1.2429686
INFO:tensorflow:global_step/sec: 56.256

Step:  32787
G_loss:  1.0052997
INFO:tensorflow:global_step/sec: 56.3638

Step:  32884
D_loss:  1.2703112
INFO:tensorflow:global_step/sec: 56.2075

Step:  32981
G_loss:  0.8927488
INFO:tensorflow:global_step/sec: 56.1118

Step:  33078
D_loss:  0.87690187
INFO:tensorflow:global_step/sec: 56.2081

Step:  33175
G_loss:  0.9771506
INFO:tensorflow:global_step/sec: 55.8746

Step:  33272
D_loss:  0.97073007
INFO:tensorflow:global_step/sec: 56.2763

Step:  33369
G_loss:  1.1380746
INFO:tensorflow:global_step/sec: 56.1579

Step:  33466
D_loss:  1.1248848
INFO:tensorflow:global_step/sec: 56.2635

Step:  33563
G_loss:  0.9612433
INF

INFO:tensorflow:global_step/sec: 56.4265

Step:  43263
G_loss:  0.8582371
INFO:tensorflow:global_step/sec: 56.2755

Step:  43360
D_loss:  1.2591661
INFO:tensorflow:global_step/sec: 55.7925

Step:  43457
G_loss:  0.9267787
INFO:tensorflow:global_step/sec: 56.437

Step:  43554
D_loss:  1.2747157
INFO:tensorflow:global_step/sec: 53.6381

Step:  43651
G_loss:  0.8427451
INFO:tensorflow:global_step/sec: 50.7234

Step:  43748
D_loss:  1.409301
INFO:tensorflow:global_step/sec: 50.8116

Step:  43845
G_loss:  0.8610372
INFO:tensorflow:global_step/sec: 50.5232

Step:  43942
D_loss:  1.1044683
INFO:tensorflow:global_step/sec: 51.0165

Step:  44039
G_loss:  0.9000694
INFO:tensorflow:global_step/sec: 50.7072

Step:  44136
D_loss:  1.1959655
INFO:tensorflow:global_step/sec: 54.3715

Step:  44233
G_loss:  0.9124857
INFO:tensorflow:global_step/sec: 56.5133

Step:  44330
D_loss:  1.55354
INFO:tensorflow:global_step/sec: 56.3895

Step:  44427
G_loss:  0.8726121
INFO:tensorflow:global_step/sec: 56.2688



...

INFO:tensorflow:global_step/sec: 55.7689

Step:  949243
G_loss:  0.87308687
INFO:tensorflow:global_step/sec: 56.2049

Step:  949340
D_loss:  1.2924073
INFO:tensorflow:global_step/sec: 56.0703

Step:  949437
G_loss:  0.85347664
INFO:tensorflow:global_step/sec: 55.7481

Step:  949534
D_loss:  1.3049464
INFO:tensorflow:global_step/sec: 56.1666

Step:  949631
G_loss:  0.9566023
INFO:tensorflow:global_step/sec: 55.7774

Step:  949728
D_loss:  1.2059057
INFO:tensorflow:global_step/sec: 56.1564

Step:  949825
G_loss:  0.95678675
INFO:tensorflow:global_step/sec: 55.8808

Step:  949922
D_loss:  1.2192345
INFO:tensorflow:global_step/sec: 55.8255

Step:  950019
G_loss:  0.95823514
INFO:tensorflow:global_step/sec: 55.9556

Step:  950116
D_loss:  1.1362468
INFO:tensorflow:global_step/sec: 55.8744

Step:  950213
G_loss:  0.8375812
INFO:tensorflow:global_step/sec: 55.9538

Step:  950310
D_loss:  1.3837578
INFO:tensorflow:global_step/sec: 56.1003

Step:  950407
G_loss:  0.84867376
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 56.1275

Step:  960010
D_loss:  0.9992968
INFO:tensorflow:global_step/sec: 56.1427

Step:  960107
G_loss:  0.96831477
INFO:tensorflow:global_step/sec: 56.2365

Step:  960204
D_loss:  1.3724926
INFO:tensorflow:global_step/sec: 56.2234

Step:  960301
G_loss:  0.89608496

Step:  960398
D_loss:  1.0678118
INFO:tensorflow:global_step/sec: 55.6459

Step:  960495
G_loss:  1.0163834
INFO:tensorflow:global_step/sec: 55.9587

Step:  960592
D_loss:  1.3021643
INFO:tensorflow:global_step/sec: 55.9604

Step:  960689
G_loss:  0.9581174
INFO:tensorflow:global_step/sec: 56.1082

Step:  960786
D_loss:  1.2524292
INFO:tensorflow:global_step/sec: 55.8889

Step:  960883
G_loss:  0.87866145
INFO:tensorflow:global_step/sec: 55.8645

Step:  960980
D_loss:  1.0293934
INFO:tensorflow:global_step/sec: 55.9981

Step:  961077
G_loss:  0.9560231
INFO:tensorflow:global_step/sec: 56.1753

Step:  961174
D_loss:  1.2297251
INFO:tensorflow:global_step/sec: 55.8291

Step:  961271
G_loss:


Step:  970777
G_loss:  0.96749276
INFO:tensorflow:global_step/sec: 56.2893

Step:  970874
D_loss:  1.2499171
INFO:tensorflow:global_step/sec: 56.293

Step:  970971
G_loss:  0.9916957
INFO:tensorflow:global_step/sec: 56.3493

Step:  971068
D_loss:  1.3924859
INFO:tensorflow:global_step/sec: 56.3022

Step:  971165
G_loss:  0.9358791
INFO:tensorflow:global_step/sec: 56.3722

Step:  971262
D_loss:  1.0858135
INFO:tensorflow:global_step/sec: 56.2233

Step:  971359
G_loss:  0.9768202
INFO:tensorflow:global_step/sec: 56.0889

Step:  971456
D_loss:  1.0734627
INFO:tensorflow:global_step/sec: 56.1587

Step:  971553
G_loss:  0.8362593
INFO:tensorflow:global_step/sec: 56.308

Step:  971650
D_loss:  1.2097119
INFO:tensorflow:global_step/sec: 56.064

Step:  971747
G_loss:  1.0778483
INFO:tensorflow:global_step/sec: 55.7703

Step:  971844
D_loss:  1.1127102
INFO:tensorflow:global_step/sec: 56.2401

Step:  971941
G_loss:  0.8427061
INFO:tensorflow:global_step/sec: 55.9998

Step:  972038
D_loss:  1.4


Step:  981447
G_loss:  0.9441498
INFO:tensorflow:global_step/sec: 55.8971

Step:  981544
D_loss:  1.2438586
INFO:tensorflow:global_step/sec: 56.1764

Step:  981641
G_loss:  0.9873652
INFO:tensorflow:global_step/sec: 56.3721

Step:  981738
D_loss:  1.025801
INFO:tensorflow:global_step/sec: 56.3614

Step:  981835
G_loss:  1.0223669
INFO:tensorflow:global_step/sec: 56.2407

Step:  981932
D_loss:  1.3067273
INFO:tensorflow:global_step/sec: 56.2807

Step:  982029
G_loss:  0.9581786
INFO:tensorflow:global_step/sec: 56.1733

Step:  982126
D_loss:  1.2231011
INFO:tensorflow:global_step/sec: 54.7786

Step:  982223
G_loss:  0.91100144
INFO:tensorflow:global_step/sec: 55.9674

Step:  982320
D_loss:  1.2407978
INFO:tensorflow:global_step/sec: 56.3393

Step:  982417
G_loss:  0.9915897
INFO:tensorflow:global_step/sec: 56.1511

Step:  982514
D_loss:  1.4411981
INFO:tensorflow:global_step/sec: 56.2849

Step:  982611
G_loss:  1.156337
INFO:tensorflow:global_step/sec: 56.0882

Step:  982708
D_loss:  1.

INFO:tensorflow:global_step/sec: 56.3721

Step:  992311
G_loss:  0.80593884
INFO:tensorflow:global_step/sec: 56.4405

Step:  992408
D_loss:  1.1965917
INFO:tensorflow:global_step/sec: 56.3134

Step:  992505
G_loss:  0.94876045
INFO:tensorflow:global_step/sec: 56.0039

Step:  992602
D_loss:  1.2854582

Step:  992699
G_loss:  1.0174484
INFO:tensorflow:global_step/sec: 56.2503

Step:  992796
D_loss:  1.3767878
INFO:tensorflow:global_step/sec: 56.0618

Step:  992893
G_loss:  0.9017976
INFO:tensorflow:global_step/sec: 56.2888

Step:  992990
D_loss:  1.3465755
INFO:tensorflow:global_step/sec: 56.2159

Step:  993087
G_loss:  1.1203315
INFO:tensorflow:global_step/sec: 56.0257

Step:  993184
D_loss:  1.0523427
INFO:tensorflow:global_step/sec: 56.2357

Step:  993281
G_loss:  0.92000747
INFO:tensorflow:global_step/sec: 56.09

Step:  993378
D_loss:  1.2114662
INFO:tensorflow:global_step/sec: 56.1285

Step:  993475
G_loss:  0.8896025
INFO:tensorflow:global_step/sec: 56.1504

Step:  993572
D_loss:  